In [9]:
import pandas as pd
import numpy as np
import random
import re
import time, pickle
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup

In [30]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                     'cow', 'diningtable', 'dog', 'horse',
                     'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']


object_text = {'aeroplane':'An airplane or aeroplane (informally plane) is a fixed-wing aircraft that is propelled forward by thrust from a jet engine, propeller, or rocket engine. Airplanes come in a variety of sizes, shapes, and wing configurations. The broad spectrum of uses for airplanes includes recreation, transportation of goods and people, military, and research. Worldwide, commercial aviation transports more than four billion passengers annually on airliners',
              'bicycle': 'A bicycle, also called a pedal cycle, bike or cycle, is a human-powered or motor-powered assisted, pedal-driven, single-track vehicle, having two wheels attached to a frame, one behind the other. A bicycle rider is called a cyclist, or bicyclist.',
              'bird':'Birds are a group of warm-blooded vertebrates constituting the class Aves, characterised by feathers, toothless beaked jaws, the laying of hard-shelled eggs, a high metabolic rate, a four-chambered heart, and a strong yet lightweight skeleton. Birds live worldwide and range in size from the bee hummingbird to the ostrich. There are about ten thousand living species, more than half of which are passerine, or "perching" birds. Birds have wings whose development varies according to species; the only known groups without wings are the extinct moa and elephant birds.',
              'boat':'A boat is a watercraft of a large range of types and sizes, but generally smaller than a ship, which is distinguished by its larger size, shape, cargo or passenger capacity, or its ability to carry boats. Small boats are typically found on inland waterways such as rivers and lakes, or in protected coastal areas. However, some boats, such as the whaleboat, were intended for use in an offshore environment. In modern naval terms, a boat is a vessel small enough to be carried aboard a ship',
              'bottle':'A bottle is a narrow-necked container made of an impermeable material (such as glass, plastic or aluminium) in various shapes and sizes that stores and transports liquids. Its mouth, at the bottling line, can be sealed with an internal stopper, an external bottle cap, a closure, or induction sealing',
              'bus':'A bus (contracted from omnibus, with variants multibus, motorbus, autobus, etc.) is a public transport road vehicle designed to carry significantly more passengers than the average cars or vans. Buses can have a capacity as high as 300 passengers, although the average bus usually carries between 30 to 100.',
              'car':'A car (or automobile) is a wheeled motor vehicle used for transportation. Most definitions of cars say that they run primarily on roads, seat one to eight people, have four wheels, and mainly transport people rather than goods.',
              'cat': 'The cat is a domestic species of small carnivorous mammal. It is the only domesticated species in the family Felidae and is often referred to as the domestic cat to distinguish it from the wild members of the family. A cat can either be a house cat, a farm cat or a feral cat; the latter ranges freely and avoids human contact. Domestic cats are valued by humans for companionship and their ability to kill rodents. About 60 cat breeds are recognized by various cat registries.',
              'chair': 'One of the basic pieces of furniture, a chair is a type of seat. Its primary features are two pieces of a durable material, attached as back and seat to one another at a 90°-or-slightly-greater angle, with usually the four corners of the horizontal seat attached in turn to four legs—or other parts of the seats underside attached to three legs or to a shaft about which a four-arm turnstile on rollers can turn—strong enough to support the weight of a person who sits on the seat (usually wide and broad enough to hold the lower body from the buttocks almost to the knees) and leans against the vertical back (usually high and wide enough to support the back to the shoulder blades).',
              'cow': 'Cattle are large, domesticated, cloven-hooved herbivores. They are a prominent modern member of the subfamily Bovinae and the most widespread species of the genus Bos. Dairy breeds are referred to as cows and non-dairy breeds are referred to as bulls.',
              'diningtable': 'The dining table , understandably, is the focal point of the dining room. It sets the overall mood of the space: a lightly stained wooden table for four exudes an informal, casual feel, while a formal oblong table for 10 will evoke sophistication and formality. A glass table feels edgy without being visually heavy, and a bistro table is perfect for a small patio.',
              'dog':'The dog or domestic dog (Canis familiaris or Canis lupus familiaris) is a domesticated descendant of the wolf which is characterized by an upturning tail. The dog is derived from an ancient, extinct wolf, and the modern wolf is the dogs nearest living relative. The dog was the first species to be domesticated, by hunter–gatherers over 15,000 years ago, before the development of agriculture.',
              'horse': 'The horse is a domesticated, odd-toed, hoofed mammal. It belongs to the taxonomic family Equidae and is one of two extant subspecies of Equus ferus. The horse has evolved over the past 45 to 55 million years from a small multi-toed creature, Eohippus, into the large, single-toed animal of today.',
              'motorbike': 'A motorcycle, often called a motorbike, bike, cycle, or (if three-wheeled) trike, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies.',
              'person':'A person (plural people or persons) is a being that has certain capacities or attributes such as reason, morality, consciousness or self-consciousness, and being a part of a culturally established form of social relations such as kinship, ownership of property, or legal responsibility. The defining features of personhood and, consequently, what makes a person count as a person, differ widely among cultures and contexts',
              'pottedplant':'A houseplant is a plant that is grown indoors in places such as residences and offices, mainly for decorative purposes, but studies have also shown them to have positive psychological effects.',
              'sheep':'Sheep are domesticated, ruminant mammals typically kept as livestock. Although the term sheep can apply to other species in the genus Ovis, in everyday usage it almost always refers to domestic sheep. Like all ruminants, sheep are members of the order Artiodactyla, the even-toed ungulates. Numbering a little over one billion, domestic sheep are also the most numerous species of sheep. An adult female is referred to as a ewe',
              'sofa':'A couch, also known as a sofa, settee, or chesterfield (see Etymology below), is a cushioned item of furniture for seating multiple people (although it is not uncommon for a single person to use a couch). It is commonly found in the form of a bench, with upholstered armrests, and often fitted with springs and tailored cushion and pillows. Although a couch is used primarily for seating, it may be used for sleeping.',
              'train':'In rail transport, a train is a series of connected vehicles that run along a railway track and transport people or freight. The word train comes from the Old French trahiner, derived from the Latin trahere meaning "to pull, to draw". Trains are typically pulled or pushed by locomotives (often known simply as "engines"), though some are self-propelled, such as multiple units. Passengers and cargo are carried in railroad cars, also known as wagons. Trains are designed to a certain gauge, or distance between rails. Most trains operate on steel tracks with steel wheels, the low friction of which makes them more efficient than other forms of transport.',
              'tvmonitor':'A television set or television receiver, more commonly called the television, TV, TV set, tube, telly, or tele, is a device that combines a tuner, display, and loudspeakers, for the purpose of viewing and hearing television broadcasts, or using it as a computer monitor. Introduced in the late 1920s in mechanical form, television sets became a popular consumer product after World War II in electronic form, using cathode ray tube (CRT) technology.'
              
              
              }

In [35]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [39]:
models = ['bert-base-uncased', 'distilbert-base-uncased-finetuned-sst-2-english', 'textattack/roberta-base-SST-2','roberta-base', 'google/electra-base-discriminator', 'xlnet-base-cased', 'xlm-roberta-base']
model_num = 0
tokenizer = AutoTokenizer.from_pretrained(models[model_num])

In [40]:
MAX_LENGTH = 300

embeddings = []
for label in object_categories:
    encoded_dict = tokenizer.encode_plus(
                            clean_text(object_text[label]),                      # Sentence to encode.
                            truncation=True,
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
    embeddings.append(encoded_dict['input_ids'])

embeddings = torch.cat(embeddings, dim=0)
print (embeddings.shape)

torch.Size([20, 300])


In [41]:
with open('bert-base.pkl', 'wb') as pkl:
    pickle.dump(np.array(embeddings), pkl)

In [42]:
with open('bert-base.pkl', 'rb') as pkl:
    doc_embedding = pickle.load(pkl)
    print (doc_embedding.shape)

(20, 300)
